In [25]:
import pandas as pd
import numpy as np
import re
import kaggle
import zipfile

In [2]:
!kaggle datasets download -d kanchana1990/ebay-iphone-pricing-trends-2023 --force


  0%|          | 0.00/26.7k [00:00<?, ?B/s]
100%|██████████| 26.7k/26.7k [00:00<00:00, 1.05MB/s]


In [3]:
zipname = 'ebay-iphone-pricing-trends-2023.zip'
with zipfile.ZipFile(zipname, 'r') as file:
    file.extractall()

In [4]:
xr = pd.read_csv('ebay_iphone_xr.csv')
pm11 = pd.read_csv('ebay_iphone_11_pro_max.csv')
pm12 = pd.read_csv('ebay_iphone_12_pro_max.csv')
pm13 = pd.read_csv('ebay_iphone_13_pro_max.csv')
pm14 = pd.read_csv('ebay_iphone_14_pro_max.csv')

In [5]:
xr['Model'] = 'iPhone XR'
pm11['Model'] = 'iPhone 11 Pro Max'
pm12['Model'] = 'iPhone 12 Pro Max'
pm13['Model'] = 'iPhone 13 Pro Max'
pm14['Model'] = 'iPhone 14 Pro Max'

In [6]:
rawdata = pd.concat([xr, pm11, pm12, pm13, pm14], ignore_index=True)
rawdata['Storage'] = rawdata['Title'].str.extract(r'(\d+GB)', expand=False)
rawdata = rawdata.dropna(subset=['Storage'])

In [7]:
rawdata['Price'] = rawdata['Price'].replace('[\\$,]', '', regex=True)
rawdata['Price'] = rawdata['Price'].apply(lambda x: np.mean([float(i) for i in x.split(' to ')] if ' to ' in x else float(x) if x else 0))

In [8]:
print(rawdata['Model'].value_counts())

Model
iPhone 14 Pro Max    777
iPhone XR            472
iPhone 12 Pro Max    258
iPhone 11 Pro Max    181
iPhone 13 Pro Max    151
Name: count, dtype: int64


In [9]:
rawdata['Storage'] = rawdata['Title'].str.extract(r'(\d+GB)', expand=False)
rawdata = rawdata.dropna(subset=['Storage'])
rawdata['Storage'] = rawdata['Storage'].replace('126GB', '128GB')
rawdata['Storage'] = rawdata['Storage'].str.rstrip('GB').astype(int)

In [10]:
print(rawdata.groupby(['Model', 'Storage'])['Price'].mean())

Model              Storage
iPhone 11 Pro Max  64          410.640654
                   256         427.392143
                   512         508.100625
iPhone 12 Pro Max  128         533.751856
                   256         513.125714
                   512         661.270800
iPhone 13 Pro Max  128         605.947778
                   256         738.140217
                   512         730.290435
iPhone 14 Pro Max  128         893.857791
                   256         941.617907
                   512        1135.416857
                   1024        900.000000
iPhone XR          64          209.097823
                   128         378.877778
                   256         423.527853
Name: Price, dtype: float64


In [12]:
release_dates = {
    'iPhone XR': '2018-10-26',
    'iPhone 11 Pro Max': '2019-09-20',
    'iPhone 12 Pro Max': '2020-11-13',
    'iPhone 13 Pro Max': '2021-09-24',
    'iPhone 14 Pro Max': '2022-09-16'
}
rawdata['ReleaseDate'] = rawdata['Model'].map(release_dates)
rawdata['ReleaseDate'] = pd.to_datetime(rawdata['ReleaseDate'])

In [13]:
release_prices= {
    'iPhone XR': {64: 749, 128: 799, 256: 899},
    'iPhone 11 Pro Max': {64: 1099, 256: 1249, 512: 1449},
    'iPhone 12 Pro Max': {128: 1099, 256: 1199, 512: 1399},
    'iPhone 13 Pro Max': {128: 1099, 256: 1199, 512: 1399, 1024: 1599},
    'iPhone 14 Pro Max': {128: 1099, 256: 1199, 512: 1399, 1024: 1599}
}
rawdata['ReleasePrice'] = rawdata.apply(lambda row: release_prices[row['Model']][row['Storage']], axis=1)

In [14]:
rawdata['DiscountPerYear'] = (rawdata['ReleasePrice'] - rawdata['Price']) / ((2023 - rawdata['ReleaseDate'].dt.year) + 1)

In [24]:
print(rawdata.groupby(['Model', 'Storage'])['DiscountPerYear'].mean())

Model              Storage
iPhone 11 Pro Max  64         137.671869
                   256        164.321571
                   512        188.179875
iPhone 12 Pro Max  128        141.312036
                   256        171.468571
                   512        184.432300
iPhone 13 Pro Max  128        164.350741
                   256        153.619928
                   512        222.903188
iPhone 14 Pro Max  128        102.571104
                   256        128.691047
                   512        131.791571
                   1024       349.500000
iPhone XR          64          89.983696
                   128         70.020370
                   256         79.245358
Name: DiscountPerYear, dtype: float64
